### Linear Regression using Gradient Descent
Victor Ernoult
<br>

Here we implement the Gradient Descent optimization process to find the best coefficients for m & b in a linear regression context. We compare it to the results we find by fitting a least square regression.

The data used is taken from one of Kaggle's competition, in which participants are asked to predict house prices based on their features. We will fit a linear regression between the liveable area of a house and its price. 

Kaggle Link: https://www.kaggle.com/c/house-prices-advanced-regression-techniques

In [62]:
import numpy as np 
import pandas as pd 

In [63]:
train = pd.read_csv("../input/train.csv")
train = train[["GrLivArea", "SalePrice"]]
train.describe()

,GrLivArea,SalePrice
count,1460.000000,1460.000000
mean,1515.463699,180921.195890
std,525.480383,79442.502883
min,334.000000,34900.000000
25%,1129.500000,129975.000000
50%,1464.000000,163000.000000
75%,1776.750000,214000.000000
max,5642.000000,755000.000000


### OLS Regression

We first fit a traditional least squares linear regression to compare the coefficients and the error with the results of the gradient descent optimization. 

In [64]:
# Fitting a Linear Regression the normal way
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

lr = LinearRegression()
X, y = (np.array(train.iloc[:,0]).reshape(-1, 1), train.iloc[:,1])
lr.fit(X, y)

print('m: ', lr.coef_[0], 'b: ',lr.intercept_)
print("MAE :", mean_absolute_error(y, lr.predict(X)))

m:  107.13035896582517 b:  18569.02585648728
MAE : 37638.72898759625


### Linear Regression using Gradient Descent

In [65]:
def gradient_descent(data, iters, learning_rate, display_progress = False):
    """ Assumes a DataFrame will be fed in, with 2 columns, x & y in that order."""
    m_curr = b_curr = 0
    n = len(data)
    x = data.iloc[:,0]
    y = data.iloc[:,1]
    
    for i in range(iters):
        y_pred = m_curr*x + b_curr
        
        # Partial derivatives of m & b
        md = -(2/n) * sum(x*(y-y_pred))
        bd = -(2/n) * sum(y-y_pred)
        
        # Adjusting the coefficients with derivatives (subject to the learning rate)
        m_curr = m_curr - learning_rate * md
        b_curr = b_curr - learning_rate * bd
        
        mae = np.mean(abs(y - (m_curr*x + b_curr)))
        if display_progress:
            print("Iteration", i, ":", "m =", m_curr, "// b =",b_curr, "// MAE =", mae)
        
    return (m_curr, b_curr, mae)

m, b, mae = gradient_descent(train, 1000, 0.0000001, False)
print("m :", m, "b :", b)
print("MAE :", mae)

m : 118.06882468668596 b : 0.4678990795538888
MAE : 38382.12418660609


The coefficients found through this method differ from the least square regression. However, we find a similar error, hinting that the method has not fully converged yet. It is interesting to note that though m quickly finds its optimal value and keeps it, b keeps increasing, but very slowly. This is surely due to the low learning rate, which was necessary to allow convergence on this particular dataset. 

### Comparison on simulated data

In [66]:
# Comparison of the 2 approaches with simulated data.
from random import uniform
size = 100
simple_data_y = [x*20+uniform(0, 5) for x in range(size)]
simple_data_x = np.array(range(size)).reshape(-1, 1)

# Least Square
print("Least Square Regression")
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.fit(simple_data_x, simple_data_y)
print('m: ', lr.coef_[0], 'b: ',lr.intercept_)
print("MAE :", mean_absolute_error(simple_data_y, lr.predict(simple_data_x)))

# Gradient Descent
print("\nGradient Descent")
data = pd.DataFrame(data = {"x" : range(size), "y" : simple_data_y})
m, b, mae = gradient_descent(data, 1000, 0.00001, False)
print("m :", m, "b :", b)
print("MAE :", mae)

Least Square Regression
m:  20.00302941500719 b:  2.596154219244454
MAE : 1.182739819267736

Gradient Descent
m : 20.037440795010948 b : 0.3137090537794922
MAE : 1.4331081988278935


Same conclusion here, gradient descent's results come rather close to OLS, both in terms of coefficients & error. As expected, OLS remains superior and much more time-effective.